In [1]:
%pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

  Using cached pyasn1-0.6.1-py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
    --------------------------------------- 0.3/13.3 MB ? eta -:--:--
   --- ------------------------------------ 1.0/13.3 MB 3.6 MB/s eta 0:00:04
   ----- ---------------------------------- 1.8/13.3 MB 3.6 MB/s eta 0:00:04
   ------- -------------------------------- 2.6/13.3 MB 3.6 MB/s eta 0:00:03
   --------- ------------------------------ 3.1/13.3 MB 3.3 MB/s eta 0:00:04
   ----------- ---------------------------- 3.7/13.3 MB 3.3 MB/s eta 0:00:03
   ------------ --------------------------- 4.2/13.3 MB 3.2 MB/s eta 0:00:03
   --------------- ------------------------ 5.2/13.3 MB 3.4 MB/s eta 0:00:03
   ------------------ --------------------- 6.3/13.3 MB 3.7 MB/s eta 0:00:02
   --------------------- ------------------ 7.1/13.3 MB 3.7 MB/s eta 0:00:02
   ----------------------- ---------------- 7.9/13.3 MB 3.6 MB/s eta 0:00:02
   ------------------------

In [ ]:
import sys
import os
from threading import Thread
from deep_translator import GoogleTranslator
from langdetect import detect
import pandas as pd
from pymongo import MongoClient
import openai
from dotenv import load_dotenv
from apscheduler.schedulers.background import BackgroundScheduler
from datetime import datetime
import pytz
from twilio.rest import Client
from __future__ import print_function
import datetime
import os.path
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/calendar.events']

def get_calendar_service():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)

        # Save the token for future use
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('calendar', 'v3', credentials=creds)
    return service

In [ ]:
def get_upcoming_events(n=5):
    service = get_calendar_service()
    now = datetime.datetime.now().isoformat() + 'Z'
    events_result = service.events().list(
        calendarId='primary', timeMin=now,
        maxResults=n, singleEvents=True,
        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print("No upcoming events found.")
        return []
    else:
        for event in events:
            start = event['start'].get('dateTime', event['start'].get('date'))
            print(f"{start} - {event['summary']}")
        return events

events = get_upcoming_events(3)
for e in events:
    print(e['summary'], e['start'])

In [ ]:
load_dotenv(override=True)
MONGODB_URL=os.getenv('MONGO_URI')
OPENAI_KEY=os.getenv('OPENAI_KEY')

def prompt_init(question):
    PROMPT = f'''
    You are a scheduler assistant. Your main task is to help manage user's schedule.
    You will be given an instruction by the user to either add an event to the calendar or retrieve events from the calendar.

    - If user asks to add an event, you need to respond with USER FORMAT:
        1. The event name
        2. The event date and time
        3. The event location
        4. The event description
        6. The event participants
    
    Then ask the user to confirm the event details. If the user confirms, you will respond SYSTEM FORMAT:
        'add_event: {
            "name": event_name,
            "date": event_date,
            "location": event_location,
            "description": event_description,
            "participants": event_participants
        }'

    If the user doesn't confirm, you will ask user to specify the correct details, revise the event details, return the corrected USER FORMAT, and ask for confirmation again (repeat the process until user confirms).
    If after three revision attempts the user doesn't confirm, you will respond with "Event not added" and stop the process.

    - If user asks to retrieve events, you need to respond "Retrieve Events" of the today and tomorrow with format:
    'retrieve_event: today_date - tomorrow_date'
    
    Question: {question}
    Answer:
    '''
    return PROMPT

def init_llm(prompt, question):
    prompt = prompt_init(question)
    openai.api_key = OPENAI_KEY
    llm = openai.chatCompletion.create(
        model="gpt-4-vision-preview",
        messages=[{
            'role': 'user',
            'content': [
                {
                    "type": "text",
                    "text": prompt
                },
                {
                    "type": "image_url",
                    "text": question
                }
            ]
        }],
        temperature=0
    )
    print(f'####### full response: {llm}')
    response = llm['choices'][0]['message']['content']
    return response

def summarize_event(prompt, question):
    instruction = init_llm(prompt, question)
    if 'add_event' in instruction:
        event_details = instruction.split('add_event: ')[1].split('}')[0] + '}'
        event_details = eval(event_details)
        event_name = event_details['name']
        start_date = event_details['start_date']
        end_date = event_details['end_date']
        event_location = event_details['location']
        event_description = event_details['description']
        event_participants = event_details['participants']
        participants = []
        for guest in event_participants:
            participants.append({"email": guest})

        return {
            'action': 'add_event',
            'event': {
                'name': event_name,
                'start_date': start_date,
                'end_date': end_date,
                'location': event_location,
                'description': event_description,
                'participants': participants
            }
        }
    elif 'retrieve_event' in instruction:
        start_date = instruction.split('retrieve_event: ')[1].split('-')[0].strip()
        end_date = instruction.split('retrieve_event: ')[1].split('-')[1].strip()
        return {
            'action': 'retrieve_event',
            'event': {
                'start_date': start_date,
                'end_date': end_date
            }
        }


